In [ ]:
data = pd.read_csv("./Movie_Clean.csv")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
import ast
import math
import calplot

from collections import Counter
from os import path
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

Wordclouds for All Genres:

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
overview_string = ' '.join(str(item) for item in data['overview'])

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def get_clean_words(text):
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]
    words = [w for w in words if w.isalpha()]
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if len(w) > 1]
    return words

In [ ]:
bow = get_clean_words(overview_string)

In [ ]:
wcString2 = ' '.join(bow)
wc = wordcloud.WordCloud(max_font_size=200, background_color = 'white', width = 2000, height = 1000).generate(wcString2)
plt.figure(figsize=(20, 10))
plt.imshow(wc, interpolation="bilinear")
plt.title("All Genres Overview WordCloud")

Wordclouds for Individual Genres:

In [ ]:
overview_dict = {}
for idx in data.index:
    genre_list = ast.literal_eval(data.at[idx, 'genres'])
    over = data.at[idx, 'overview']
    if over == over:
        for n in genre_list:
            if n in overview_dict:
                overview_dict[n] = overview_dict[n] + ' ' + over
            else:
                overview_dict[n] = over

In [ ]:
for key, value in overview_dict.items():
    value = get_clean_words(value)

In [ ]:
for key, value in overview_dict.items():
    wc = wordcloud.WordCloud(max_font_size=200, background_color = 'white', width = 2000, height = 1000).generate(value)
    plt.figure(figsize=(20, 10))
    plt.imshow(wc, interpolation="bilinear")
    plt.title("Genre: %s" % key)

Sentiment Analysis based on Genre:

In [ ]:
for key, value in overview_dict.items():
    sia = SentimentIntensityAnalyzer()
    print(key)
    print(sia.polarity_scores(value))

In [ ]:
overview_dict2 = {}
for idx in data.index:
    genre_list = ast.literal_eval(data.at[idx, 'genres'])
    over = data.at[idx, 'overview']
    if over == over:
        for n in genre_list:
            if n in overview_dict2:
                overview_dict2[n].append(over)
            else: 
                overview_dict2[n] = [over]sentiment_dict = {}

In [ ]:
sentiment_dict = {}
for key in overview_dict2.keys():
    sentiment_dict[key + '_pos'] = []
    sentiment_dict[key + '_neu'] = []
    sentiment_dict[key + '_neg'] = []
    for n in overview_dict2[key]:
        scores = sia.polarity_scores(n)
        sentiment_dict[key + '_pos'].append(scores['pos'])
        sentiment_dict[key + '_neu'].append(scores['neu'])
        sentiment_dict[key + '_neg'].append(scores['neg'])

In [ ]:
def sent_plot(color, name):
    plt.hist(sentiment_dict[name], color = color, edgecolor = 'black')
    avg = np.nanmean(sentiment_dict[name])
    plt.axvline(x=avg, color='red', linestyle='--', linewidth=3, label='Average: ' + str(round(avg, 4)))
    plt.legend()
    plt.title(name + ' Sentiment Scores')
    plt.savefig(name + '.png')
    plt.clf()

In [ ]:
for key in overview_dict2.keys():
    name = key + '_pos'
    sent_plot('green', name)
    name = key + '_neu'
    sent_plot('orange', name)
    name = key + '_neg'
    sent_plot('blue', name)